# Calculando Holm-Bonferroni para modelos

## Importação 

### Importando pacotes

In [10]:
# Importando pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

### Importando dados

In [11]:
# Importando resultados 
results = pd.read_excel('extracao_final_avaliacao2.xlsx', sheet_name='Dados')

## Calculando Holm-Bonferroni

In [13]:
# Força conversão para float, erros viram NaN
results['VARIAVEL_EXOGENA_ESTATISTICA'] = pd.to_numeric(results['VARIAVEL_EXOGENA_ESTATISTICA'], errors='coerce')

# Verifique se há valores não numéricos
if results['VARIAVEL_EXOGENA_ESTATISTICA'].isna().any():
    print("⚠️ Existem valores não numéricos na coluna VARIAVEL_EXOGENA_ESTATISTICA que foram convertidos para NaN.")

⚠️ Existem valores não numéricos na coluna VARIAVEL_EXOGENA_ESTATISTICA que foram convertidos para NaN.


In [15]:
# Calcular p-value bicaudal assumindo distribuição normal padrão
results['p_value'] = 2 * (1 - norm.cdf(np.abs(results['VARIAVEL_EXOGENA_ESTATISTICA'])))

In [17]:
# Aplicar correção Holm-Bonferroni
alpha = 0.05
n = len(results)

# Ordenar os p-values em ordem crescente
results_sorted = results.sort_values('p_value').reset_index(drop=False)
results_sorted['holm_threshold'] = alpha / (n - results_sorted.index)

# Verificar significância corrigida
results_sorted['rejeita_H0'] = results_sorted['p_value'] <= results_sorted['holm_threshold']

# Criar coluna com texto de significância
results_sorted['significancia'] = np.where(
    results_sorted['rejeita_H0'],
    'significativo',
    'não significativo'
)

# Identificar o novo limiar de p-value corrigido
if results_sorted['rejeita_H0'].any():
    cutoff_p = results_sorted.loc[results_sorted['rejeita_H0'], 'p_value'].max()
    print(f'Novo limiar de p-value (Holm-Bonferroni): {cutoff_p:.6f}, para {n} testes.')
else:
    print('Nenhum teste significativo após Holm-Bonferroni.')

Novo limiar de p-value (Holm-Bonferroni): 0.000136, para 604 testes.


In [20]:
# Seleciona apenas as colunas criadas
colunas_exportar = ["UF", "AGRAVO", 'p_value', 'holm_threshold', 'rejeita_H0', 'significancia']
results_export = results_sorted[colunas_exportar]

# Exporta para Excel
results_export.to_excel('resultados_correcoes.xlsx', index=False)

Para avaliar a significância estatística da variável exógena em 604 modelos SARIMA ajustados via SARIMAX, foi extraída a estatística t associada a essa variável em cada modelo. Assumindo aproximação pela distribuição normal padrão — válida para grandes amostras — os p-values bicaudais foram estimados a partir dos valores absolutos das estatísticas t. Para controlar o erro tipo I decorrente dos múltiplos testes realizados, foi aplicada a correção de Holm-Bonferroni ao conjunto de p-values. Como resultado, foi definido um novo limiar de significância ajustado de 0.000136, abaixo do qual os efeitos da variável exógena são considerados estatisticamente significativos após correção.

To evaluate the statistical significance of an exogenous covariate across 604 SARIMA models fitted via the SARIMAX procedure, the t-statistic corresponding to the covariate coefficient was extracted from each model. Under the assumption of large-sample properties, these test statistics were approximated as following a standard normal distribution, and two-sided p-values were computed accordingly. To adjust for multiple comparisons and control the family-wise error rate, the Holm-Bonferroni procedure was applied to the resulting p-values. This yielded an adjusted significance threshold of 0.000136, below which associations were deemed statistically significant after correction for multiplicity.